_Observação geral: Esse guia rápido supõe que a abordagem utilizada é sequencial e não pela Keras API_

## Plot de gráfico
Forma de chamada: 

~~~python
graph_plots(argumento)
~~~

Isso irá te dar um gráfico do treino e da validação. Dá pra usar a função para qualquer métrica, mas o gráfico exibirá com o nome de acc.

*Nota: Não usar para conjuntos apenas de treino e sem validação*

In [1]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10,7]
plt.style.use('seaborn-whitegrid')
def graph_plots(history):
    plt.clf()
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)
    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.figure()
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.show()

## Callbacks

As callbacks são especificações para alguns comportamentos do modelo, como:

* A ReduceLROnPlateau, que diminui a taxa de aprendizado conforme a perda não minimiza, para assim caminhar em espaço menor e não ficar presa em um mínimo local;
* A ModelCheckpoint, que salva num arquivo .h5 (Formato que o Keras sabe ler) a melhor versão do seu modelo;
* E a EarlyStopping, que, ao contrário da ReduceLROnPlateau, para o treinamento quando ele parecer inútil.

_Observação: **NÃO é uma boa ideia utilizar a ReduceLR sem o par do ModelCheckpoint**_

A callbacks_list_v1 contém o par __ReduceLR__ e __ModelCheckpoint__, salvando a melhor versão do modelo e o otimizando para não se prender em mínimos locais de acordo com a _Validation Loss_.

A callbacks_list_v2 é a __EarlyStopping__, que parará o treinamento caso a perda não diminua em um determinado numero de épocas (default pela função: 5).

Para a callbacks_list_v1, de um nome (string) ao seu modelo (argumento name) e opcionalmente um caminho de arquivo(string), se por exemplo você quiser salvar seu modelo em uma pasta diferente da atual. O modelo se salvará no local especificado.

A callbacks_list_v2 tem o argumento opcional patience, que especifica a tolerância em épocas caso você queira que ela demore mais ou menos de 5 épocas da métrica monitorada para parar.

Ambas funções possuem o argumento opcional monitor, caso queira monitorar outra métrica se não a _Validation Loss_.

_Observação: A compilação só aceita essas callbacks em forma de lista_.

Forma de chamada:

~~~python
callbacks_list = callbacks_list_v1('MeuModelo')
##ou
callbacks_list = callbacks_list_v2()
~~~

Forma de compilação do modelo:

~~~python
history = model.fit(x=x,
                    y=y,
                    #...especificaçoes de fit diversas
                    callbacks=callbacks_list)
~~~

In [ ]:
from keras import callbacks
import os

def callbacks_list_v1(name, file_path='',monitor = 'val_loss'):
    callbacks_list = [
                    callbacks.ReduceLROnPlateau(
                        monitor=monitor,
                        factor=0.1,
                        patience=10,
                        ),
                    callbacks.ModelCheckpoint(
                        filepath=os.path.join(file_path,name+'.h5'),
                        monitor=monitor,
                        save_best_only=True,
                        mode="auto",
                        save_weights_only=False,
                        ),
                    ]
    return callbacks_list

In [ ]:
def callbacks_list_v2(patience=5,monitor ='val_loss'):
    callbacks_list = [
        callbacks.EarlyStopping(
                    monitor=monitor,
                    min_delta=0,
                    patience=patience,
                    verbose=0,
                    mode="auto",
                    baseline=None,
                    restore_best_weights=True,
        )
                     ]
    return callbacks_list

## Recomendação de Otimizadores

* Nos meus testes, o SGD não foi tão bom;
* O Nadam apresenta algumas vezes melhores resultados;
* O RMSProp é o mais usado, e traz sempre bons resultados;

Adapte o learning_rate. Aumente-o se quiser que o modelo aprenda mais rápido, ou diminua-o se achar que ele está indo rápido demais. Recomendo testar com 1e-3 e 1e-5

In [ ]:
from keras import optimizers

NadamOpt = optimizers.Nadam(lr=1e-4)
RMSOpt = optimizers.RMSprop(lr=1e-4)
SGDOpt = optimizers.SGD(lr=1e-4)

## Recomendação de ativações

A LeakyRelu é implementada como camada e não como uma própria ativação, então se quiser usá-la passe pelas camadas sem ativação e coloque a camada layers.LeakyRelu() após cada camada.

In [ ]:
import tensorflow as tf
from keras import layers

SwishActivation = tf.nn.swish
model.add(layers.LeakyRelu())

## Dica Final

Aplicar normalização no seu modelo pode melhorá-lo. Se quiser, após cada camada convolucional, adicione a camada BatchNormalization da seguinte forma:
 
~~~python
model.add(layers.BatchNormalization())
~~~